# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.
* After installing gmaps, to show the figure: run "jupyter nbextension enable --py --sys-prefix gmaps"

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
# from IPython.display import display
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

#### In this homework, metric system has been used for measurement and calculation.

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

# Study data files
weather_path = "weather_data.csv"

# Read the mouse data and the study results
weather_data = pd.read_csv(weather_path)
print(len(weather_data))
weather_data.head()

467


,Unnamed: 0,city id,city,country,latitude,longitute,temperature(C),humidity(%),wind speed(mps),cloud(%),receive date
0,0,2350886,Aku,NG,6.7081,7.3167,21.76,98,1.64,100,15-06-2022 11:16:02
1,1,3622716,Nicoya,CR,10.1483,-85.4520,22.56,98,1.37,100,15-06-2022 11:13:00
2,2,4030556,Rikitea,PF,-23.1203,-134.9692,22.87,70,3.95,96,15-06-2022 11:16:03
3,3,3440777,Rocha,UY,-34.4833,-54.3333,9.82,92,2.71,56,15-06-2022 11:12:26
4,4,358620,Safaga,EG,26.7292,33.9365,29.07,40,8.81,0,15-06-2022 11:16:04


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = weather_data[["latitude", "longitute"]]
rating = weather_data["humidity(%)"].astype(float)

# Plot Heatmap
gmaps.configure(api_key=g_key)
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=2)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Ideal weather condition:
# A temperature range from 20C (68F) to 27C (80F)
# Wind speed less then 4.5 meter/second (10mph)
# Zero clouds

# df.drop(df[(df['Unit_Price'] >400) & (df['Unit_Price'] < 600)].index, inplace=True)
hotel_data = weather_data.drop(weather_data[((weather_data["temperature(C)"] < 20 )
                                 | (weather_data["temperature(C)"] > 27) 
                                 | (weather_data["wind speed(mps)"] > 4.5)
                                 | ((weather_data["cloud(%)"] > 0)))
                                 ].index, inplace=False)
print(f"hotel data: {len(hotel_data)} rows")
hotel_data.head()
hotel_data.dropna(inplace = True) 

hotel data: 16 rows


### Hotel Map
* Store into variable named `hotel_data'.
* Add Hotel name, address, rating columns to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:

hotel_data["hotel name"] = ""
hotel_data["hotel address"] = ""
hotel_data["hotel rating"] = 0
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
key_word = "hotel"
type = "lodging"
radius = 5000

for index, row in hotel_data.iterrows():

    lat = row["latitude"]
    lon = row["longitute"] 
    target_coordinates = f"{lat},{lon} "

    # set up a parameters dictionary    
    params = {
        "location": target_coordinates,
        "keyword": "hotel",
        "type": type,
        "radius": radius,
        "key": g_key}

    # Run a request using our params dictionary
    response = requests.get(base_url, params=params).json()

    # Extract results
    results = response['results']
    pprint(response)

    # Get the first operational hotel from the json results
    for i in results:
        if results[0]['business_status'] == 'OPERATIONAL' :
            break 

    # Save hotel name and address   
    try:
        print(f"Hotel found: {results[0]['name']} ***********")
        hotel_data.loc[index, 'hotel name'] = results[0]['name']
        hotel_data.loc[index, 'hotel address'] = results[0]['vicinity']
        hotel_data.loc[index, 'hotel rating'] = results[0]['rating']
        
 #
        
    except (KeyError, IndexError):        
        print("Missing field/result... skipping.")
        
    print("*" * 100 + "\n\n")


{'html_attributions': [],
 'next_page_token': 'Aap_uECTn22p9vVkRGsjDamXWLp_QJeYEEd7Mw5YKa-iRkLyP8fzMiw1hBnzDiQLutV0_U-6cJ70EBFTTSCqVK-Z7xS6zt1kL4I0pa9st0oI0CwoHD1fr6jyVHUqlG9jz5Ceu7qTzlffSob0PI33M9igM528IhJxOkZj-Z5mn7LrfoCH0NVG5MLDvRXHZ-u2XHH7PRB5La0LLyD1iFjfTCLct58AQYDEmyi3yEyW3vn_4xxG4GGltc0um3e9wVe_cjUpzqM_eWHb_9aCR5jZzVjNszblf_ab3lO0M8_QY6szmu9Mjr51aKEh06BQu8ZFFEZWHy3VOn20rd0D7Yxtwgdg62eSw6fqVsMKoyxUUuApTflfrrtcN49LqV9VOVc8eLMkNkPwMmNQVkCoVMt8BlQhztJq4lEYnlWeEgxeBFgoVc9NVasWF9pBdm4vj9IUmYBTig',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -20.7259501, 'lng': 139.4925696},
                           'viewport': {'northeast': {'lat': -20.72473882010728,
                                                      'lng': 139.4938258798927},
                                        'southwest': {'lat': -20.72743847989273,
                                                      'lng': 139.4911262201072}}},
              'icon': 'https://maps.gstat

In [6]:

print(len(hotel_data))
hotel_data.head(20)


16


,Unnamed: 0,city id,city,country,latitude,longitute,temperature(C),humidity(%),wind speed(mps),cloud(%),receive date,hotel name,hotel address,hotel rating
53,53,2065594,Mount Isa,AU,-20.7333,139.5000,23.87,20,3.09,0,15-06-2022 11:16:39,ibis Styles Mt Isa Verona,"Cnr Rodeo Dr And, Camooweal St, Mount Isa",3.9
140,140,293842,Kiryat Gat,IL,31.6100,34.7642,21.76,72,1.01,0,15-06-2022 11:17:42,Desert Gat,"Sderot HaAtsma'ut 134, Kiryat Gat",4.8
170,170,2077895,Alice Springs,AU,-23.7000,133.8833,20.68,32,0.51,0,15-06-2022 11:18:05,Stay at Alice Springs Hotel (formerly Aurora A...,"11 Leichhardt Terrace, Alice Springs",3.5
183,183,1160571,Khāsh,IR,28.2211,61.2158,24.73,43,1.12,0,15-06-2022 11:18:15,مهمانپذیر مسجد حضرت ابوالفضل (ع),656W+FW7، خاش،، استان سیستان و بلوچستان،,0.0
229,229,137956,Dārāb,IR,28.7519,54.5444,26.66,39,1.02,0,15-06-2022 11:18:49,Naghshe shapoor Hotel,"Darab, Daneshjoo Boulevard، Pasdaran Blv., QH6...",4.3
231,231,5962442,Goderich,CA,43.7501,-81.7165,25.18,47,2.06,0,15-06-2022 11:18:50,Samuels Hotel,"34031 Saltford Rd, Goderich",4.5
263,263,3172394,Naples,IT,40.8333,14.2500,22.74,71,1.03,0,15-06-2022 11:19:16,"Holiday Inn Naples, an IHG Hotel","Via Domenico Aulisio, Centro Direzionale Isola...",4.0
283,283,88533,Awjilah,LY,29.1081,21.2869,22.89,66,4.38,0,15-06-2022 11:19:32,,,0.0
299,299,921786,Mitsamiouli,KM,-11.3847,43.2844,25.01,73,3.93,0,15-06-2022 11:19:44,Al Camar Lodge,RN 1,4.3
348,348,1090467,Boueni,YT,-12.9025,45.0761,21.99,72,2.62,0,15-06-2022 11:20:25,LE CHISSIOUA,"CCD 5, Tahiti Plage. Route Nationale, Sada",4.0


In [7]:
# Drop cities that have no hotel found

hotel_data= hotel_data[hotel_data["hotel name"] != ""]
print(len(hotel_data))

# Store latitude and longitude in locations
locations = hotel_data[["latitude", "longitute"]]
rating = hotel_data["hotel rating"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=(rating * 300) , 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

15


Figure(layout=FigureLayout(height='420px'))

In [8]:
hotel_locations = hotel_data[["latitude", "longitute"]]

# locations = hotel_data[["latitude", "longitute"]]
info_box_template = """
<dl><dl>
<dt>Name:</dt><dd>{hotel name}</dd>
<dt>City:</dt><dd>{city}</dd>
<dt>Country:</dt><dd>{country}</dd>
<dt>Rating:</dt><dd>{hotel rating}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_data.iterrows()]

marker_layer = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

## 